# 01 - Join property components

Merges property component data supplied by the Department of Revenue to produce one-line-per-geocode (i.e. PropertyNumber) data tables with estimated tax figures for 2022 and 2023 tax rolls. Those tables are then joined for geocodes in both years of data to produce a data table that allows year-to-year changes in existing property to be analyzed.

In [1]:
# Libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

Import data provided by MTDOR in Oct. 2023. This data breaks individual properties into their taxable components (different components are sometimes in different property classes, meaning they are subject to different tax rates).

In [2]:
dtype = {
    'TaxYear': str,
    'Co': str,
    'LevyDistrictCode': str,
    'PropertyNumber': str,
    'TaxClass': str,
    'ClassCode': str,
    'ClassCodeDesc': str,
    'AbateInd': str,
    'SM': str,
    'PropertyTypeDesc': str,
    'TIFName': str,
    'TIFCode': str,
    'AssessmentCode': str,
    'NameLast': str,
    'Address1': str,
    'Address2': str,
    'Address3': str,
    'City': str,
    'State': str,
    'ZIP': str,
    'Situs_Address': str,
    'Situs_City': str,
    'Situs_State': str,
    'Situs_ZipCode': str,
    'MV': float,
    'TV': float,
    'Mills': float
}
property_components_22 = pd.read_csv('./raw/TY2022_Property_Data.csv', dtype=dtype, encoding='ISO-8859-1')
property_components_23 = pd.read_csv('./raw/TY2023_Property_Data.csv', dtype=dtype, encoding='ISO-8859-1')

/var/folders/gk/_q4g8knj7354h1qcc0gsdcq00000gp/T/ipykernel_21232/3768125689.py:31: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  property_components_23 = pd.read_csv('./raw/TY2023_Property_Data.csv', dtype=dtype, encoding='ISO-8859-1')


Clean and process property component data.

Cleaning involves adding county names to data for ease of analysis and filling in missing property numbers for Stillwater Mining Company properties.

Processing steps adjust each component's millage rate to include a consistent 95 state mills in accordance with November's Montana Supreme Court ruling. Estimated tax values are calculated for each component using taxable value and the resulting millage rate.

We're also assigning a simplified type category to each property to allow for property-type-based analysis that uses five simple property categories as opposed to the ~20 in the department's data.


In [3]:
# DATA CLEANING

# Assign county names based on county number
COUNTIES = {"1":"SILVER BOW","2":"CASCADE","3":"YELLOWSTONE","4":"MISSOULA","5":"LEWIS AND CLARK","6":"GALLATIN","7":"FLATHEAD","8":"FERGUS","9":"POWDER RIVER","10":"CARBON","11":"PHILLIPS","12":"HILL","13":"RAVALLI","14":"CUSTER","15":"LAKE","16":"DAWSON","17":"ROOSEVELT","18":"BEAVERHEAD","19":"CHOUTEAU","20":"VALLEY","21":"TOOLE","22":"BIG HORN","23":"MUSSELSHELL","24":"BLAINE","25":"MADISON","26":"PONDERA","27":"RICHLAND","28":"POWELL","29":"ROSEBUD","30":"DEER LODGE","31":"TETON","32":"STILLWATER","33":"TREASURE","34":"SHERIDAN","35":"SANDERS","36":"JUDITH BASIN","37":"DANIELS","38":"GLACIER","39":"FALLON","40":"SWEET GRASS","41":"MCCONE","42":"CARTER","43":"BROADWATER","44":"WHEATLAND","45":"PRAIRIE","46":"GRANITE","47":"MEAGHER","48":"LIBERTY","49":"PARK","50":"GARFIELD","51":"JEFFERSON","52":"WIBAUX","53":"GOLDEN VALLEY","54":"MINERAL","55":"PETROLEUM","56":"LINCOLN"}
property_components_22['County'] = property_components_22['Co'].map(lambda x: COUNTIES[str(x)])
property_components_23['County'] = property_components_23['Co'].map(lambda x: COUNTIES[str(x)])

# Add missing geocode for Stillwater Mining Company properties to keep grouping code from breaking later
#   Appending PropertyTypeDesc here to maintain a single PropTypeDesc for each PropertyNumber
property_components_22['PropertyNumber'].fillna('mtfp_smc_' + property_components_22['PropertyTypeDesc'], inplace=True)
property_components_23['PropertyNumber'].fillna('mtfp_smc_' + property_components_23['PropertyTypeDesc'], inplace=True)

In [4]:
# DATA PROCESSING

# Adjust  Mills values to account for Nov. 2023 MT Supreme Court ruling on state mills
#   Source data factored in 77.89 mills for counties that levied the lower rate
#   This adjustment should set every property to a millage rate that includes the full 95
STATE_MILLS = pd.read_csv('./config/counties-95-mill-status-oct-23.csv')
STATE_MILLS['County'] = STATE_MILLS['County'].str.upper() # For data merge
property_components_23 = property_components_23.merge(STATE_MILLS, left_on='County', right_on='County')
property_components_23['Mills_unadjusted'] = property_components_23['Mills']
# Set Mills for all 2023 components to millage value with full 95 collected
#   Math: Strip off initial state millage component then add back in a full 95
property_components_23['Mills'] = property_components_23['Mills'] - property_components_23['state_mills'] + 95 

# Calculate estimated taxes amount for each property component

def calc_estimated_taxes (df):
    MILL_FACTOR = 1000
    return df['TV'] * df['Mills'] / MILL_FACTOR # TV is property component taxable value

property_components_22['Est_Taxes'] = calc_estimated_taxes(property_components_22)
property_components_23['Est_Taxes'] = calc_estimated_taxes(property_components_23)

# Assign simplified property categories
SIMPLE_CATEGORIES = {
    # Residential
    'Residential Property': 'Residential',
    
    # Commercial
    'Commercial Property': 'Commercial',
    'Mixed Use - Res & Comm': 'Commercial',
    
    # Industrial
    'Industrial Real Property': 'Industrial',
    'Industrial Personal Property Attached to Real Property': 'Industrial',
    'Industrial Personal Property not Attached to Real': 'Industrial',
    'Centrally Assessed GenTax': 'Industrial',
    'Locally Assessed Utility': 'Industrial',
    
    # Ag
    'Agricultural and Timber Properties': 'Agricultural',
    
    # Other
    'Net and Gross Proceeds': 'Other',
    'Non-Qualified Ag': 'Other', # Sounds like this is ag-like property that hasn't qualified for ag rates
    'Personal Property - Strict Personal': 'Other',
    'Personal Property Attached to Real Property': 'Other',
    'Government Property': 'Other',
    'DNRC Cabin Site - Govt': 'Other',
    'Tribal Properties': 'Other',
    'Manufactured Homes not Attached to Real': 'Other', # Excluding manufactured homes from residential
    'Manufactured Homes Attached to Real': 'Other', # Excluding manufactured homes from residential
    'Condo/Townhouse Master': 'Other'
}
property_components_22['PropertyCat_mtfp'] = property_components_22['PropertyTypeDesc'].map(lambda x: SIMPLE_CATEGORIES[x])
property_components_23['PropertyCat_mtfp'] = property_components_23['PropertyTypeDesc'].map(lambda x: SIMPLE_CATEGORIES[x])

Group property components for each year by `PropertyNumber`, or Geocode, summing `MV` (market value), `TV` (taxable value) and `Est_Tax` (estimated taxes) values to produce a total value for each property.

We're grouping by `PropertyNumber` rather than `AssessmentCode` because most centrally assessed properties don't have an `AssessmentCode`. Property numbers for Stillwater Mining Company were missing from the source data and added above.

Grouped data tables are then merged with an inner join, producing a joined data table that includes properties present in both years of the data. This should keep newly taxable property from influencing the results.

In [5]:
def groupByProperty(df):
    def concat (x):
        uniques = [str(d) for d in list(x.unique())]
        if len(uniques) == 0: return ''
        if len(uniques) == 1: return uniques[0] # For computational efficiency
        uniques.sort()
        return  ', '.join(uniques)
    return df.groupby([
        'County',
        'PropertyNumber',
        'PropertyTypeDesc', # Constant for property numbers
        'PropertyCat_mtfp',
        'NameLast',
    ], group_keys=False).agg({
        'TaxClass': concat,
        'ClassCodeDesc': concat,
        'MV': 'sum',
        'TV': 'sum',
        'Est_Taxes': 'sum',
    })

grouped22 = groupByProperty(property_components_22).reset_index()
grouped23 = groupByProperty(property_components_23).reset_index()


In [6]:
# Join properties present in both years of data
joined = grouped23.merge(grouped22, 
    how='inner', 
    left_on=['County','PropertyNumber'], 
    right_on=['County', 'PropertyNumber'],
    suffixes=['_23','_22'],
)

# Change analyses
joined['MV_change'] = joined['MV_23'] - joined['MV_22']
joined['MV_per_change'] = (joined['MV_23'] - joined['MV_22']) / joined['MV_22']
joined['Est_Taxes_change'] = joined['Est_Taxes_23'] - joined['Est_Taxes_22']
joined['Est_Taxes_per_change'] = joined['Est_Taxes_change'] / joined['Est_Taxes_22']

In [7]:
# Merge overlap check
#   both = properties present in both years of data
#   left_only = properties present in 2023 data, not 2022
#   right_only = properties present in 2022 data, not 2023
grouped23.merge(grouped22, 
    how='outer', 
    left_on=['County','PropertyNumber'], 
    right_on=['County', 'PropertyNumber'],
    suffixes=['_23','_22'],
    indicator=True,
)['_merge'].value_counts()

_merge
both          955896
left_only      13509
right_only      6755
Name: count, dtype: int64

Write output files for further analysis

In [8]:
joined.to_csv('./processed/joined-on-geocode.csv', index=False)
property_components_23.to_csv('./processed/2023-property-components.csv', index=False)
property_components_22.to_csv('./processed/2022-property-components.csv', index=False)